

Chúng ta sẽ thảo luận về giao thức HTTP sử dụng thư viện Requests, một phương pháp phổ biến để xử lý giao thức HTTP trong Python. Chúng ta sẽ xem xét:

- Tổng quan về thư viện Requests của Python để làm việc với giao thức HTTP.
- Giới thiệu về các yêu cầu GET và POST.

#### Thư viện Requests:
Requests là một thư viện Python cho phép bạn dễ dàng gửi các yêu cầu HTTP/1.1. Bạn có thể import thư viện như sau:
```python
import requests
```

#### Ví dụ về yêu cầu GET:
Bạn có thể thực hiện một yêu cầu GET đến `www.ibm.com` như sau:
```python
response = requests.get('https://www.ibm.com')
print(response.status_code)  # In ra mã trạng thái (200 cho OK)
```
- Sử dụng `response.headers` để xem tiêu đề phản hồi.
- Sử dụng `response.text` để hiển thị nội dung HTML trong thân phản hồi.

#### Yêu cầu POST:
Khác với GET, yêu cầu POST gửi dữ liệu trong phần thân yêu cầu (request body) thay vì trong URL. Để thực hiện yêu cầu POST, bạn thay đổi endpoint và sử dụng phương thức `post()` như sau:
```python
payload = {'name': 'Joseph', 'ID': '123'}
response = requests.post('https://httpbin.org/post', data=payload)
print(response.text)
```
- Kết quả của POST không chứa các cặp tên-giá trị trong URL, mà chúng được gửi trong phần thân yêu cầu.

---



Chúng ta sẽ tìm hiểu về Web Scraping. Sau khi xem xong, bạn sẽ có thể:

- Định nghĩa khái niệm web scraping;
- Hiểu vai trò của các đối tượng BeautifulSoup;
- Áp dụng phương thức `find_all`;
- Thực hiện web scraping trên một trang web.

#### Web Scraping là gì?
Web scraping là quá trình tự động trích xuất thông tin từ một trang web. Thay vì tốn hàng giờ để sao chép thủ công, bạn có thể hoàn thành công việc trong vài phút với một chút mã Python và sự trợ giúp từ hai module: Requests và BeautifulSoup.

#### Ví dụ:
Giả sử bạn cần tìm tên và lương của các cầu thủ trong Giải bóng rổ quốc gia từ một trang web.  
1. **Nhập BeautifulSoup**:  
   ```python
   from bs4 import BeautifulSoup
   ```
2. **Phân tích HTML**:  
   Lưu HTML của trang web dưới dạng chuỗi và sử dụng BeautifulSoup để phân tích:
   ```python
   soup = BeautifulSoup(HTML, 'html.parser')
   ```

#### Phân tích cấu trúc:
BeautifulSoup đại diện cho HTML dưới dạng các đối tượng cây (Tree-like objects).  
- Ví dụ: Để truy cập thẻ `<title>`, bạn có thể sử dụng:
  ```python
  title = soup.title
  ```

#### Áp dụng `find_all`:
Phương thức `find_all` lọc và trả về tất cả các thẻ phù hợp:
- Ví dụ: Lấy tất cả các hàng trong bảng HTML:
  ```python
  rows = soup.find_all('tr')
  for row in rows:
      cells = row.find_all('td')
      for cell in cells:
          print(cell.text)
  ```

#### Kết hợp với Requests:
Để lấy dữ liệu HTML từ một trang web:
```python
import requests
response = requests.get('https://example.com')
soup = BeautifulSoup(response.text, 'html.parser')
```
Sau đó, bạn có thể sử dụng BeautifulSoup để duyệt và trích xuất thông tin từ trang HTML đó. Hãy thử thực hành trong các bài lab.

### OBJECTIVE

### Sau khi hoàn thành phòng thí nghiệm này, bạn sẽ có thể:

- Sử dụng các thư viện **requests** và **BeautifulSoup** để trích xuất nội dung của một trang web.
- Phân tích mã HTML của một trang web để tìm thông tin liên quan.
- Trích xuất thông tin liên quan và lưu nó theo định dạng yêu cầu.


### SCENARIO

Hãy giả sử rằng bạn đã được một tổ chức quản lý rạp chiếu phim thuê để trích xuất thông tin về 50 bộ phim có đánh giá trung bình tốt nhất từ liên kết web được chia sẻ dưới đây:

[https://web.archive.org/web/20230902185655/https://en.everybodywiki.com/100_Most_Highly-Ranked_Films](https://web.archive.org/web/20230902185655/https://en.everybodywiki.com/100_Most_Highly-Ranked_Films)

Thông tin yêu cầu bao gồm:
- **Xếp hạng trung bình (Average Rank)**
- **Tên phim (Film)**
- **Năm sản xuất (Year)**

Bạn cần viết một script Python có tên là `webscraping_movies.py` để trích xuất thông tin này và lưu nó vào một tệp CSV có tên `top_50_films.csv`. Đồng thời, bạn cũng cần lưu cùng thông tin đó vào cơ sở dữ liệu SQLite có tên là `Movies.db`, trong bảng dữ liệu có tên là `Top_50`.


In [10]:
#import library
import pandas as pd 
import requests
import sqlite3
from bs4 import BeautifulSoup

### Khởi tạo các thực thể đã biết
Bạn cần khai báo một số thực thể ngay từ đầu. Ví dụ:
- URL cần thiết
- Tên tệp CSV để lưu bản ghi
- Tên cơ sở dữ liệu và tên bảng để lưu trữ bản ghi
- Các thực thể cần được lưu trữ

Ngoài ra, vì bạn chỉ cần lấy 50 kết quả đầu tiên, bạn sẽ cần một biến đếm vòng lặp được khởi tạo bằng `0`. Bạn có thể khởi tạo tất cả những điều này bằng đoạn mã sau trong tệp `webscraping_movies.py`:


In [11]:
# url trang web
url = "https://web.archive.org/web/20230902185655/https://en.everybodywiki.com/100_Most_Highly-Ranked_Films"

# dia chi tep csv luu ban ghi
csv_path = r"D:\IBM-Data-Engineering-Python-Project-for-Data-Engineering\Web-Scraping\top_50_films.csv"

# ten co so du lieu de luu ban ghi
db_name = "Movies.db"

# ten bang de luu ban ghi
table_name = "Top_50"

# khoi tao dataframe
df = pd.DataFrame(columns=["Average Rank", "Film", "Year"])
count = 0


### Tải trang web để Web Scraping
Để truy cập thông tin cần thiết từ trang web, trước tiên bạn cần tải toàn bộ trang web dưới dạng tài liệu HTML trong Python bằng cách sử dụng hàm `requests.get().text`. Sau đó, phân tích cú pháp văn bản này dưới định dạng HTML bằng **BeautifulSoup** để có thể trích xuất thông tin liên quan.


In [12]:
#tai trang web duoi dang tai lieu HTML
html_page = requests.get(url).text
#su dung thu vien BeautifulSoup de phan tich HTML
data = BeautifulSoup(html_page, 'html.parser')

### Trích xuất thông tin cần thiết
Để trích xuất thông tin phù hợp từ trang web, bạn cần viết vòng lặp để duyệt qua các hàng của bảng dữ liệu. Các hàng của bảng có thể được truy cập bằng cách sử dụng hàm `find_all()` với đối tượng **BeautifulSoup** như sau:




In [13]:
#lay tat cac doi tuong duoc luu tru bang the 'tbody'
tables = data.find_all('tbody')

#Inspect web thay bang luu tru cac bo phim nam o the 'tbody' dau tien 
# Cac hang trong bang duoc luu tru bang the 'tr'
rows = tables[0].find_all('tr')

### Chức năng của đoạn mã như sau:

1. **Duyệt qua nội dung của biến `rows`**:
   - Lặp qua từng hàng trong bảng đã lấy được.

2. **Kiểm tra biến đếm vòng lặp để giới hạn 50 mục nhập**:
   - Đảm bảo chỉ trích xuất 50 hàng đầu tiên.

3. **Trích xuất tất cả các đối tượng dữ liệu `<td>` trong hàng và lưu chúng vào biến `col`**:
   - Lấy dữ liệu từ các cột trong hàng hiện tại.

4. **Kiểm tra độ dài của `col`**:
   - Nếu độ dài của `col` là 0 (không có dữ liệu trong hàng hiện tại), bỏ qua hàng đó.
   - Điều này rất quan trọng vì đôi khi có những hàng được hợp nhất (merged rows) không hiển thị rõ ràng trên giao diện trang web.

5. **Tạo một từ điển `data_dict`**:
   - Từ điển này có các khóa trùng với các cột của DataFrame được tạo để lưu trữ đầu ra trước đó.
   - Giá trị của các khóa được lấy từ ba cột đầu tiên trong dữ liệu.

6. **Chuyển đổi từ điển thành DataFrame và nối nó vào DataFrame hiện có**:
   - Cách này giúp dữ liệu được thêm dần vào DataFrame trong mỗi lần lặp.

7. **Tăng giá trị của biến đếm vòng lặp**:
   - Cập nhật biến đếm để theo dõi số hàng đã xử lý.

8. **Dừng lặp khi biến đếm đạt đến 50**:
   - Khi đã trích xuất đủ 50 hàng, ngừng vòng lặp và thoát.



In [14]:
#duyet qua tat ca cac the 'td' de lay thong tin ve bo phim
# the 'th' chi chua cac truong tieu de nhu Average Rank, Film, Year
for row in rows:
    if count<50:
        col = row.find_all('td')
        if len(col)!=0:
            data_dict = {"Average Rank": col[0].contents[0],
                         "Film": col[1].contents[0],
                         "Year": col[2].contents[0]}
            df1 = pd.DataFrame(data_dict, index=[0]) #khoi tao df1 la dataframe chua 1 ban ghi, index = [0] de xac dinh rang df1 chi co 1 ban ghi
            df = pd.concat([df,df1], ignore_index=True)
            count+=1
    else:
        break

print(df)

   Average Rank                                           Film  Year
0             1                                  The Godfather  1972
1             2                                   Citizen Kane  1941
2             3                                     Casablanca  1942
3             4                         The Godfather, Part II  1974
4             5                            Singin' in the Rain  1952
5             6                                         Psycho  1960
6             7                                    Rear Window  1954
7             8                                 Apocalypse Now  1979
8             9                          2001: A Space Odyssey  1968
9            10                                  Seven Samurai  1954
10           11                                        Vertigo  1958
11           12                                    Sunset Blvd  1950
12           13                                   Modern Times  1936
13           14                   

### STORING DATA

In [15]:
df.to_csv(csv_path)


Để lưu trữ dữ liệu cần thiết vào cơ sở dữ liệu, trước tiên bạn cần khởi tạo kết nối tới cơ sở dữ liệu, lưu dataframe dưới dạng một bảng, sau đó đóng kết nối

### Ý nghĩa các tham số trong câu lệnh to_sql

- **`name`**: Tên bảng trong cơ sở dữ liệu.
  - Đây là tên bảng mà dữ liệu từ DataFrame sẽ được lưu vào.
  - Trong đoạn mã của bạn, `table_name` là tên của bảng đó.

- **`con`**: Kết nối cơ sở dữ liệu.
  - Là đối tượng kết nối tới cơ sở dữ liệu, trong trường hợp này là `conn` (kết nối SQLite được tạo bằng `sqlite3.connect`).

- **`if_exists`**: Xử lý nếu bảng đã tồn tại trong cơ sở dữ liệu.
  - Có các tùy chọn:
    - `'fail'`: Mặc định, sẽ tạo lỗi nếu bảng đã tồn tại.
    - `'replace'`: Xóa bảng cũ (nếu có) và tạo bảng mới, sau đó lưu dữ liệu vào bảng này.
    - `'append'`: Thêm dữ liệu mới vào bảng hiện có, không thay thế bảng.
- **`index`**: Chỉ định có lưu cột index của DataFrame vào bảng hay không.

  - `'index=True'`, cột index của DataFrame sẽ được lưu như một cột trong bảng.
  - `'index=False'`, index của DataFrame sẽ bị bỏ qua khi lưu.


In [16]:
# khoi tao ket noi toi co so du lieu
conn = sqlite3.connect(db_name)

df.to_sql(table_name, conn, if_exists='replace', index=False)
conn.close()

### Bài tập thực hành

Hãy thử các bài tập thực hành sau để kiểm tra hiểu biết của bạn về nội dung đã học trong bài. Lưu ý rằng lời giải cho các bài tập sau **sẽ không được chia sẻ**. Bạn được khuyến khích tham gia các diễn đàn thảo luận nếu cần hỗ trợ.

1. Chỉnh sửa mã để trích xuất các cột **Film**, **Year**, và **Rotten Tomatoes' Top 100**.

2. Giới hạn kết quả chỉ bao gồm **25 mục đầu tiên**.

3. Lọc đầu ra để chỉ in các bộ phim được phát hành trong thập kỷ 2000 (bao gồm năm 2000).
